In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

%matplotlib inline

In [9]:
def readAndShowImageAndFilter(image,filtr):
    im = util.img_as_bool(io.imread(image, as_gray=True))
    im_filtr = util.img_as_bool(io.imread(filtr, as_gray=True))
    io.imshow_collection([im,im_filtr], cmap='gray')
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Wireboard':'/home/jim-alexander/Dokumenter/bildebehandling/Bilder/image_resources/circuitmask.tif',
        'Balls With Reflection':'/home/jim-alexander/Dokumenter/bildebehandling/Bilder/image_resources/balls-with-reflections.tif',
        'binary2':'binary2.pbm'
    },
    description='Bilde: ',
    disabled=False,
)

inputFilter = widgets.Dropdown(
    options={
        'Cross':'se_cross.pbm',
        'Box':'boxSE.png'
    },
    description='Filter: ',
    disabled=False,
)

interactive(readAndShowImageAndFilter, image=inputImage, filtr=inputFilter)

interactive(children=(Dropdown(description='Bilde: ', options={'Wireboard': '/home/jim-alexander/Dokumenter/bi…

In [17]:
def erode(image,se):
    se_height,se_width = se.shape

    pad_v = se_height//2
    pad_h = se_width//2

    padded = util.pad(image,(pad_v,pad_h),mode="constant")

    out = numpy.zeros(image.shape,dtype=bool)
    for i in range(image.shape[0]):
      for j in range(image.shape[1]):
        subimage = padded[i:i+se_height,j:j+se_width]
        out[i,j] = numpy.array_equal(subimage*se,se)
    
    return out

def dilation(image, se):
    se_height,se_width = se.shape

    pad_v = se_height//2
    pad_h = se_width//2

    padded = util.pad(image,(pad_v,pad_h),mode="constant")

    out = numpy.zeros(image.shape,dtype=bool)
    for i in range(image.shape[0]):
      for j in range(image.shape[1]):
        subimage = padded[i:i+se_height,j:j+se_width]
        out[i,j] = numpy.sum(subimage*se)>0 
    return out

def opening(image, se):
    return dilation(erode(image,se),se)

def closing(image, se):
    return erode(dilation(image,se),se)

def boundryExtract(image, se):
    return image ^ erode(image,se)

def holeFilling(image, se):
    image_iter = numpy.zeros(image.shape)
    image_iter[260,150] = True
    
    while True: 
        image_next = dilation(image_iter,se) & (~image)
        
        if(numpy.array_equal(image_iter,image_next)):
            break
        
        image_iter = image_next
    
    return image_next | image

In [18]:
inputMethod = widgets.Dropdown(
    options={'erode':erode,
             'dilation':dilation,
             'opening':opening,
             'closing':closing,
             'Boundry Extract':boundryExtract,
             'Hole filling': holeFilling
            },
    description='Filter: ',
    disabled=False,
)

def update(image, filtr, method):
    im = util.img_as_bool(io.imread(image, as_gray=True))
    se = util.img_as_bool(io.imread(filtr, as_gray=True))
    
    im_prosessed = method(im,se)
    
    io.imshow_collection([im,im_prosessed], cmap='gray')
    io.show()
    
interactive(update, image=inputImage, filtr=inputFilter, method=inputMethod)

interactive(children=(Dropdown(description='Bilde: ', index=1, options={'Wireboard': '/home/jim-alexander/Doku…